# Depression Detection using classification

This file fine-tunes `bert-base-uncased` for depression detection.

To add more features, I use fine-tuned BERT model to generate sentiment scores for each sentence. The fine-tuned BERT model will classify the input sentences into `Extremely Negative`, `Negative`, `Neutral`, `Positive`, and `Extremely Positive` five classes and corresponding probability.

In [1]:
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
from ast import literal_eval
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
import torch.nn as nn
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import numpy as np

try:
    from google.colab import drive
    drive.mount('/content/gdrive')

    train_path = '/content/gdrive/MyDrive/advanced-ml-project/data/train_emotion.csv'
    test_path = '/content/gdrive/MyDrive/advanced-ml-project/data/test_emotion.csv'
    dev_path = '/content/gdrive/MyDrive/advanced-ml-project/data/dev_emotion.csv'

    eval_model_path = '/content/gdrive/MyDrive/advanced-ml-project/bert-depression-detection.pth'
except:
    train_path = 'data/train.tsv'
    test_path = 'data/test.tsv'
    dev_path = 'data/dev.tsv'

    eval_model_path = 'bert-depression-detection.pth'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cuda', index=0)

## Load dataset

In [3]:
label2idx = {'moderate': 0, 'not depression': 1, 'severe':2}
idx2label = {0: 'moderate', 1: 'not depression', 2: 'severe'}

In [4]:
data = pd.read_csv(train_path, sep='\t')
data['emotion_scores'] = data['emotion_scores'].apply(literal_eval)
data['label'] = data['label'].apply(lambda x: label2idx[x])

tmp = pd.read_csv(test_path, sep='\t')
tmp['emotion_scores'] = tmp['emotion_scores'].apply(literal_eval)
tmp['label'] = tmp['label'].apply(lambda x: label2idx[x])
data = pd.concat([data, tmp], axis=0)

tmp = pd.read_csv(dev_path, sep='\t')
tmp['emotion_scores'] = tmp['emotion_scores'].apply(literal_eval)
tmp['label'] = tmp['label'].apply(lambda x: label2idx[x])
data = pd.concat([data, tmp], axis=0)

print('Length of Data:', len(data))
print(data.label.value_counts())
data.head(10)

Length of Data: 16632
0    10494
1     4649
2     1489
Name: label, dtype: int64


,PID,text,label,emotion_scores
0,train_pid_1,Waiting for my mind to have a breakdown once t...,0,"[4.610891819000244, 0.563305139541626, -2.3326..."
1,train_pid_2,My new years resolution : I'm gonna get my ass...,0,"[-0.5888213515281677, 1.3056291341781616, -0.5..."
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,0,"[-1.0411173105239868, 2.4230809211730957, -0.0..."
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",0,"[-1.099108099937439, -1.7904443740844727, -2.0..."
4,train_pid_5,Sat in the dark and cried myself going into th...,0,"[-3.502218723297119, -1.8709688186645508, -1.1..."
5,train_pid_6,I will probably end it when my mum isn't aroun...,0,"[1.1285946369171143, 2.6311261653900146, -1.37..."
6,train_pid_7,Fuck 2019 : Left abusive relationship. Moved i...,0,"[5.702419281005859, -0.9708327054977417, -1.97..."
7,train_pid_8,I am at a new year's eve party and I want to c...,0,"[4.561304569244385, 1.2020779848098755, -2.334..."
8,train_pid_9,Death of my father : My father died in the beg...,0,"[5.312351703643799, -0.806493878364563, -1.955..."
9,train_pid_10,Empty and stuck in a loop every day : In any o...,0,"[-1.0623170137405396, -1.2411339282989502, -1...."


In [5]:
X = data[['text', 'emotion_scores']].values
y = data['label'].values
print(X.shape)
print(y.shape)

(16632, 2)
(16632,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11642, 2)
(11642,)
(4990, 2)
(4990,)


## Preprocessing

In [7]:
class MyDataSet(Dataset):
    def __init__(self, X, y, max_len=512):
        self.X = X
        self.y = y
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.max_len = max_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        text = self.X[index][0]
        label = self.y[index]
        features = self.X[index][1]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=self.max_len)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'token_type_ids': encoding['token_type_ids'].squeeze(),
            'features': torch.tensor(features),
            'label': torch.tensor(label),
        }

In [8]:
trainData = MyDataSet(X_train, y_train)
testData = MyDataSet(X_test, y_test)

train_loader = DataLoader(trainData, batch_size=5, shuffle=True)
test_loader = DataLoader(testData, batch_size=5, shuffle=True)

## Build Model

In [9]:
class DepressionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = AutoModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(self.classifier.config.hidden_size + 5, 3)
        """ Initialize the weights of linear layer."""
        nn.init.xavier_normal_(self.linear1.weight)

    def forward(self, input_ids, token_type_ids, attention_mask, features):
        output = self.classifier(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        output = output.last_hidden_state[:, 0, :] # [batch size, hidden size]
        output = self.dropout(output)
        output = torch.cat((output, features), dim=-1) # [batch size, hidden size+num extra features]
        output = self.linear1(output) # [batch size, num labels]
        return output

## Training

In [ ]:
model = DepressionClassifier().to(device)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float, device=device)

criterion = CrossEntropyLoss(weight=class_weights)
optimizer = AdamW(model.parameters(), lr=2e-5)
n_epoch = 3

In [ ]:
for epoch in tqdm(range(n_epoch), desc=f"Training progress", colour="#00ff00"):
    training_loss = []
    training_f1 = []
    testing_loss = []
    testing_f1 = []

    model.train()
    for i, inputs in enumerate(tqdm(train_loader, leave=False, desc=f"Epoch {epoch + 1}/{n_epoch}", colour="#00ff00")):
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        features = inputs['features'].to(device)
        label = inputs['label'].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, token_type_ids, attention_mask, features)

        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        y_pred = torch.argmax(logits, 1)
        f1 = f1_score(label.cpu(), y_pred.cpu(), average='weighted')

        training_loss.append(loss.item())
        training_f1.append(f1)

    model.eval()
    for i, inputs in enumerate(tqdm(test_loader, leave=False, desc="Evaluating:")):
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        features = inputs['features'].to(device)
        label = inputs['label'].to(device)

        logits = model(input_ids, token_type_ids, attention_mask, features)

        loss = criterion(logits, label)

        y_pred = torch.argmax(logits, 1)
        f1 = f1_score(label.cpu(), y_pred.cpu(), average='weighted')

        testing_loss.append(loss.item())
        testing_f1.append(f1)

    training_loss, training_f1 = np.array(training_loss), np.array(training_f1)
    testing_loss, testing_f1 = np.array(testing_loss), np.array(testing_f1)
    print('Epoch %d, training loss: %.3f, traing f1: %.3f, testing loss: %.3f, testing f1: %.3f' %
         (epoch+1, np.mean(training_loss), np.mean(training_f1), np.mean(testing_loss), np.mean(testing_f1)))

Training progress:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/3:   0%|          | 0/2329 [00:00<?, ?it/s]

Evaluating::   0%|          | 0/998 [00:00<?, ?it/s]

Epoch 1, training loss: 0.802, traing f1: 0.651, testing loss: 0.857, testing f1: 0.503


Epoch 2/3:   0%|          | 0/2329 [00:00<?, ?it/s]

Evaluating::   0%|          | 0/998 [00:00<?, ?it/s]

Epoch 2, training loss: 0.497, traing f1: 0.800, testing loss: 0.623, testing f1: 0.752


Epoch 3/3:   0%|          | 0/2329 [00:00<?, ?it/s]

Evaluating::   0%|          | 0/998 [00:00<?, ?it/s]

Epoch 3, training loss: 0.304, traing f1: 0.880, testing loss: 0.726, testing f1: 0.769


In [ ]:
torch.save(model.state_dict(), eval_model_path)

## Evaluation

In [10]:
model = DepressionClassifier().to(device)
model.load_state_dict(torch.load(eval_model_path))

<All keys matched successfully>

In [11]:
model.eval()
preds = []
y_true = []
for i, inputs in enumerate(tqdm(test_loader, leave=False, desc="Evaluating")):
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    features = inputs['features'].to(device)
    label = inputs['label'].to(device)

    logits = model(input_ids, token_type_ids, attention_mask, features)

    y_pred = torch.argmax(logits, 1)

    preds += y_pred.cpu().tolist()
    y_true += label.cpu().tolist()

f1 = f1_score(np.array(y_true), np.array(preds), average='weighted')
print(f"F1 score: {f1}")

Evaluating::   0%|          | 0/998 [00:00<?, ?it/s]

F1 score: 0.7686033377761181
